# Coding Project Analysis JEM Forms


### Data Output: Date, Specimen ID, Rig Opertator, Layer, Cell Type Prediction (Human), Patch Duration(min)

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd

import JEM_post_patch as pp #post patch script
#post_patch_column = 'extraction.postPatch'
#post_patch_pipette_column = 'extraction.endPipetteR'

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
#from single_cell_ephys.lims_funcs import limsquery
#from single_cell_ephys.file_funcs import validated_input

In [2]:
pd.options.display.max_columns = None  #Displays all the columns 

In [3]:
#Used csv from 5/25/2018
shiny_human_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_human.csv')
shiny_mouse_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_mouse.csv')

In [4]:
#List of common functions(In progress)

def gen_fil(df, col, cond):
    '''
    Filters the column with a certain condition
    df: dataframe
    col: column
    cond: condition
    '''
    filtered = df[df[col] == cond]
    return filtered

def gen_cat(df, col):
    '''
    Assigns astype to the column
    df: dataframe
    col: column
    '''    
    category = df[col].astype('category')
    return category

def cell_count(P_num):
    '''
    Counts number of Lims tube id based on P# as number of cells
    df = dataframe
    count() = counts number
    '''
    user = df1[df1['Lims tube id'].str.contains(P_num)]
    return user['Lims tube id'].count()

def P_user(P_num):
    '''
    Prints cell_count 
    '''
    print("Total cells:", cell_count(P_num))

In [5]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath)
    return json_paths

In [6]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
#json_list

In [7]:
def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

### json_df

In [8]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)

In [10]:
#Creates patch_duration column and outputs in minutes
json_df['patch_duration'] = pd.to_datetime(json_df['extraction.timeRetractionEnd']) - pd.to_datetime(json_df['recording.timeWholeCellStart'])
json_df['patch_duration'] = (json_df['patch_duration'].astype('timedelta64[s]'))/60
json_df.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.233333
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.233333
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.200000
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.833333
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.350000


### Fixing up date column & adding post patch column

In [11]:
json_df['date'] = json_df['date'].str[:10] #Strip away the time
json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD
json_df.set_index('date', inplace = True)
json_df = pp.postpatch_reclass(json_df) #Makes post patch column with the script
json_df.sort_index(inplace = True) #Sort the date column just in case

#start_date = input('Enter the start date for your dataframe (YYYY-MM-DD): ')
start_date = '2017-10-01'
json_df1 = json_df[start_date:]
json_df1 = gen_fil(json_df1, 'status', 'SUCCESS') 
json_df1.sort_index(inplace = True) #Sort the date column just in case

In [12]:
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1100,2.0 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,13:59:45,13:59:31,14:01:56,13:59:47,354,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,13:51:23,13:52:27,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_A05_M,9.483333,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1400,2.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,14:28:51,14:28:20,14:31:39,14:29:08,355,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,14:19:45,14:21:02,7,lindsayn,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171002_01_A05_M,10.616667,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Fluorescence in Pipette,no,nucleus_present,-32.0,-32.0,No Bubbles,NaN,15:43:26,15:42:58,15:45:24,15:43:27,055,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:37:03,15:38:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,6.733333,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,2,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10.2,NaN,Cell Dimmed,not_intentionally,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,13:57:41,13:57:21,14:00:00,13:58:00,451,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.06.02,NaN,NaN,Recording Time = 8 min,NaN,NaN,NaN,5.0,NaN,13:46:41,13:48:08,4,ramr,Wave of Death started after 30 min,NaN,SUCCESS,"Patch/Cell Unstable,Patch Became Leaky,Ended A...",EPhys_171002_01_A04_M,11.866667,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,4.7,NaN,NaN,no,nucleus_absent,-26.6,-26.6,NaN,NaN,15:42:11,15:41:40,15:43:57,15:42:11,405,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,15:37:45,15:38:32,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,5.416667,No-high seal


In [13]:
json_df1.tail()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-05-11,35,NaN,NaN,1700,NaN,Cell Swelled,no,nucleus_present,-38.0,-38.0,No Bubbles,NaN,11:05:44 -07:00,11:04:23 -07:00,11:08:00 -07:00,NaN,P8S4_180601_353_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.03.02,VISp5,NaN,NaN,NaN,NaN,NaN,4.1,NaN,10:57:51 -07:00,10:58:49 -07:00,2,lindsayn,NaN,Good,SUCCESS,NaN,NaN,9.183333,Nuc-high seal
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-05-11,48,NaN,NaN,3000,NaN,Cell Swelled,no,nucleus_present,-38.0,-38.0,No Bubbles,NaN,11:25:26 -07:00,11:24:01 -07:00,11:26:34 -07:00,NaN,P8S4_180601_354_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.03.02,VISp4,NaN,NaN,NaN,NaN,NaN,4,NaN,11:16:37 -07:00,11:18:01 -07:00,2,lindsayn,NaN,Good,SUCCESS,NaN,NaN,8.550000,Nuc-high seal
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,3,1.0,None,NaN,2018-05-11,59,NaN,NaN,6.9,No Alexa fill visible\nPartial-Nucleus,"Cell Dimmed,Cell Swelled",no,nucleus_present,-20.0,-20.0,No Bubbles,NaN,14:21:44 -07:00,14:21:24 -07:00,14:23:01 -07:00,NaN,P9S4_180601_407_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Th-P2A-FlpO;Ai65F-391841.04.01,VISp5,NaN,NaN,NaN,NaN,NaN,3.6,NaN,14:13:49 -07:00,14:14:46 -07:00,3,lisak,#Health,'Wave of Death',SUCCESS,NaN,NaN,8.250000,Nuc-low seal
2018-06-01,2018-05-31,NaN,NaN,4,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,4,1.0,None,NaN,2018-05-11,0,NaN,NaN,2500,NaN,Cell Dimmed,no,nucleus_absent,-20.0,-20.0,No Bubbles,NaN,10:23:43 -07:00,10:23:27 -07:00,10:24:43 -07:00,NaN,P1S4_180601_001_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.03.01,VISp2/3,NaN,NaN,NaN,NaN,NaN,3.9,NaN,10:19:29 -07:00,10:20:34 -07:00,4,kristenh,working outside WSE,Good,SUCCESS,NaN,NaN,4.150000,No-high seal
2018-06-01,2018-05-31,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,3,3.0,None,NaN,2018-05-11,47.5,NaN,NaN,1000,Alexa fill visible\nNucleated,"Fluorescence in Pipette,Cell Dimmed",no,nucleus_present,-22.0,-22.0,No Bubbles,NaN,10:38:59 -07:00,10:38:34 -07:00,10:40:49 -07:00,NaN,P9S4_180601_402_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.04.01,VISp2/3,NaN,NaN,NaN,NaN,NaN,4.3,NaN,10:32:50 -07:00,10:33:53 -07:00,3,lisak,#Health,Good,SUCCESS,"Patch/Cell Unstable,Patch Became Leaky",NaN,6.933333,Nuc-high seal


### Filter for Human & Mouse dataframe

In [14]:
json_df1['human?'] = json_df1['limsSpecName'].str.match(r"H\d\d") #Creates new boolean column

json_df2 = gen_fil(json_df1, 'human?', True) #HUMAN
json_df3 = gen_fil(json_df1, 'human?', False) #MOUSE
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1100,2.0 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,13:59:45,13:59:31,14:01:56,13:59:47,354,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,13:51:23,13:52:27,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_A05_M,9.483333,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1400,2.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,14:28:51,14:28:20,14:31:39,14:29:08,355,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,14:19:45,14:21:02,7,lindsayn,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171002_01_A05_M,10.616667,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Fluorescence in Pipette,no,nucleus_present,-32.0,-32.0,No Bubbles,NaN,15:43:26,15:42:58,15:45:24,15:43:27,055,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:37:03,15:38:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,6.733333,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,2,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10.2,NaN,Cell Dimmed,not_intentionally,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,13:57:41,13:57:21,14:00:00,13:58:00,451,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.06.02,NaN,NaN,Recording Time = 8 min,NaN,NaN,NaN,5.0,NaN,13:46:41,13:48:08,4,ramr,Wave of Death started after 30 min,NaN,SUCCESS,"Patch/Cell Unstable,Patch Became Leaky,Ended A...",EPhys_171002_01_A04_M,11.866667,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,4.7,NaN,NaN,no,nucleus_absent,-26.6,-26.6,NaN,NaN,15:42:11,15:41:40,15:43:57,15:42:11,405,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,15:37:45,15:38:32,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,5.416667,No

## HUMAN dataframe

In [15]:
json_df2 = json_df2.reset_index()
human_df = json_df2[['date',
                     'limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'recording.humanCellTypePrediction',
                     'post_patch',
                     'patch_duration',
                     'extraction.tubeID']]

human_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-10 to 2017-11-27
#manualRoi avaible data at 2017-11-27 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-27

shiny_human_df = shiny_human_df.loc[:,['sample_id',
                                    'cell_name_label',
                                    'res_index_label']]

final_H_df = pd.merge(left = human_df, 
                      right = shiny_human_df, 
                      left_on = 'extraction.tubeID', 
                      right_on = 'sample_id', 
                      how = 'inner')

In [16]:
final_H_df.head()

,date,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,patch_duration,extraction.tubeID,sample_id,cell_name_label,res_index_label
0,2017-11-27,H17.26.003.11.14,dijonh,NaN,TCx3,NaN,Nuc-high seal,23.716667,PBS4_171127_501_A01,PBS4_171127_501_A01,H17.26.003.11.14.01,1.000000
1,2017-11-27,H17.26.003.11.16,lindsayn,NaN,TCx5,Unknown Interneuron,No-high seal,23.383333,P8S4_171127_353_A01,P8S4_171127_353_A01,H17.26.003.11.16.03,0.504463
2,2017-11-27,H17.26.003.11.14,dijonh,NaN,TCx5,NaN,Nuc-high seal,22.050000,PBS4_171127_502_A01,PBS4_171127_502_A01,H17.26.003.11.14.02,1.000000
3,2017-11-27,H17.26.003.11.16,lindsayn,NaN,TCx2,Unknown Interneuron,Nuc-high seal,21.033333,P8S4_171127_354_A01,P8S4_171127_354_A01,H17.26.003.11.16.04,1.000000
4,2017-11-27,H17.26.003.11.16,lindsayn,NaN,TCx5,Unknown Interneuron,No-high seal,13.016667,P8S4_171127_351_A01,P8S4_171127_351_A01,H17.26.003.11.16.01,0.386299


In [17]:
final_H_df.tail()

,date,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,patch_duration,extraction.tubeID,sample_id,cell_name_label,res_index_label
225,2018-03-20,H18.03.005.11.09,rustym,NaN,TCx2,Pyramidal,Nuc-low seal,21.116667,P2S4_180320_057_A01,P2S4_180320_057_A01,H18.03.005.11.09.04,0.537958
226,2018-03-21,H18.03.005.11.04,ramr,NaN,TCx2,Unknown Interneuron,No-low seal,20.533333,PAS4_180321_451_A01,PAS4_180321_451_A01,H18.03.005.11.04.01,1.000000
227,2018-03-27,H18.03.007.11.05,rustym,NaN,TCx2,Pyramidal,Nuc-high seal,18.750000,P2S4_180327_056_A01,P2S4_180327_056_A01,H18.03.007.11.05.02,1.000000
228,2018-03-27,H18.03.007.11.05,rustym,NaN,TCx3,Pyramidal,Nuc-high seal,9.883333,P2S4_180327_055_A01,P2S4_180327_055_A01,H18.03.007.11.05.01,1.000000
229,2018-03-27,H18.03.007.11.05,rustym,NaN,TCx2,Pyramidal,No-high seal,25.083333,P2S4_180327_057_A01,P2S4_180327_057_A01,H18.03.007.11.05.03,0.365141


## MOUSE dataframe

In [18]:
json_df3 = json_df3.reset_index()
mouse_df = json_df3[['date',
                     'limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'post_patch',
                     'patch_duration',
                     'extraction.tubeID']]

mouse_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-02 to 2017-11-10
#manualRoi avaible data at 2017-11-10 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-10

shiny_mouse_df = shiny_mouse_df.loc[:,['sample_id',
                                       'cell_name_label',
                                       'res_index_label']]

final_M_df = pd.merge(left = mouse_df, 
                      right = shiny_mouse_df, 
                      left_on = 'extraction.tubeID', 
                      right_on = 'sample_id', 
                      how = 'inner')

In [19]:
final_M_df.head()

,date,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,patch_duration,extraction.tubeID,sample_id,cell_name_label,res_index_label
0,2017-11-08,Pvalb-IRES-Cre;Ai14-357946.02.01,aarono,NaN,VISp2/3,No-high seal,12.433333,PXS4_171108_251_A01,PXS4_171108_251_A01,Pvalb-IRES-Cre;Ai14-357946.02.01.01,0.458638
1,2017-11-10,Sst-IRES-Cre;Ai14-357932.04.02,lindsayn,NaN,VISp5,No-high seal,15.833333,P8S4_171110_357_A01,P8S4_171110_357_A01,Sst-IRES-Cre;Ai14-357932.04.02.02,0.458638
2,2017-11-10,Sst-IRES-Cre;Ai14-357932.04.01,lisak,NaN,VISp5,Entire cell,5.583333,P9S4_171110_401_A01,P9S4_171110_401_A01,Sst-IRES-Cre;Ai14-357932.04.01.01,0.329975
3,2017-11-10,Nkx2-1-CreERT2;Ai14-357183.04.02,lindsayn,NaN,VISp5,No-high seal,5.283333,P8S4_171110_351_A01,P8S4_171110_351_A01,Nkx2-1-CreERT2;Ai14-357183.04.02.01,0.329975
4,2017-11-10,Nkx2-1-CreERT2;Ai14-357183.04.01,aarono,NaN,VISp5,Nuc-high seal,10.166667,PXS4_171110_251_A01,PXS4_171110_251_A01,Nkx2-1-CreERT2;Ai14-357183.04.01.01,0.926190


In [20]:
final_M_df.tail()

,date,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,patch_duration,extraction.tubeID,sample_id,cell_name_label,res_index_label
1318,2018-04-09,Rbp4-Cre_KL100;Ai14-383389.06.02,ramr,NaN,VISp5,Nuc-low seal,9.750000,PAS4_180409_451_A01,PAS4_180409_451_A01,Rbp4-Cre_KL100;Ai14-383389.06.02.01,0.483141
1319,2018-04-10,Vip-IRES-Cre;Ai14-383085.05.01,kristenh,NaN,VISp2/3,Entire cell,13.316667,P1S4_180410_001_A01,P1S4_180410_001_A01,Vip-IRES-Cre;Ai14-383085.05.01.01,1.000000
1320,2018-04-10,Vip-IRES-Cre;Ai14-383085.03.02,rustym,NaN,VISp2/3,No-high seal,16.550000,P2S4_180410_051_A01,P2S4_180410_051_A01,Vip-IRES-Cre;Ai14-383085.03.02.01,0.562351
1321,2018-04-10,Slc32a1-IRES-Cre;Ai14-384210.03.01,kristenh,NaN,VISp2/3,No-high seal,15.733333,P1S4_180410_003_A01,P1S4_180410_003_A01,Slc32a1-IRES-Cre;Ai14-384210.03.01.01,0.688578
1322,2018-04-10,Vip-IRES-Cre;Ai14-383085.05.01,kristenh,NaN,VISp2/3,No-low seal,4.516667,P1S4_180410_002_A01,P1S4_180410_002_A01,Vip-IRES-Cre;Ai14-383085.05.01.02,0.458638


## Extra

In [21]:
human_df['post_patch'].value_counts()

Nuc-high seal    183
No-low seal      123
No-high seal     108
Nuc-low seal      49
Entire cell        3
Name: post_patch, dtype: int64

In [22]:
final_H_df['post_patch'].value_counts()

Nuc-high seal    79
No-low seal      65
No-high seal     59
Nuc-low seal     25
Entire cell       2
Name: post_patch, dtype: int64

In [23]:
mouse_df['post_patch'].value_counts()

Nuc-high seal    1450
No-high seal      536
No-low seal       448
Nuc-low seal      299
Entire cell        41
Name: post_patch, dtype: int64

In [24]:
final_M_df['post_patch'].value_counts()

Nuc-high seal    649
No-low seal      257
No-high seal     228
Nuc-low seal     171
Entire cell       18
Name: post_patch, dtype: int64

In [25]:
def nuc_count(df):
    #user = str(input('Enter the user number (P#): '))
    user = 'PA'
    
    nh = df[df['post_patch'].str.contains('Nuc-high seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nh = nh['post_patch'].count()

    nl = df[df['post_patch'].str.contains('Nuc-low seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nl = nl['post_patch'].count()

    noh = df[df['post_patch'].str.contains('No-high seal') & 
             df['extraction.tubeID'].str.contains(user)]
    noh = noh['post_patch'].count()

    nol = df[df['post_patch'].str.contains('No-low seal') & 
             df['extraction.tubeID'].str.contains(user)]
    nol = nol['post_patch'].count()

    en = df[df['post_patch'].str.contains('Entire cell') & 
            df['extraction.tubeID'].str.contains(user)]
    en = en['post_patch'].count()
    

    print 'Nucleated:', nh
    print 'Partial nucleated', nl
    print 'Outside-out:', noh
    print 'No seal:', nol
    print 'Entire cell:', en   

In [26]:
nuc_count(mouse_df)

Nucleated: 143
Partial nucleated 61
Outside-out: 10
No seal: 42
Entire cell: 8


In [27]:
nuc_count(final_M_df)

Nucleated: 93
Partial nucleated 37
Outside-out: 7
No seal: 26
Entire cell: 7


In [28]:
nuc_count(human_df)

Nucleated: 2
Partial nucleated 4
Outside-out: 2
No seal: 10
Entire cell: 0


In [29]:
nuc_count(final_H_df)

Nucleated: 2
Partial nucleated 2
Outside-out: 2
No seal: 2
Entire cell: 0


In [30]:
#Problem: index date column doesn't show dates after forming new combined df with shiny